# Setup Notebook

In [1]:
import os
import numpy as np
import glob
import time
import PIL.Image as Image
from tqdm.notebook import tqdm
from itertools import chain
from collections import OrderedDict


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import ipywidgets
from IPython import display

In [2]:
# set device
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The code will run on GPU.


In [3]:
# install weight and biases library
!pip install wandb > /dev/null
import wandb

In [4]:
# setup repo
import sys, os
if os.path.exists('./dl-computer-vision'):
    !rm -r dl-computer-vision > /dev/null
!git clone https://github.com/simonamtoft/dl-computer-vision.git > /dev/null
sys.path.append('dl-computer-vision')

from data import HORSES, ZEBRAS
from models import Discriminator, Generator
from training import train_cycle_gan

Cloning into 'dl-computer-vision'...
remote: Enumerating objects: 1058, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 1058 (delta 195), reused 207 (delta 108), pack-reused 753
Receiving objects: 100% (1058/1058), 21.24 MiB | 39.40 MiB/s, done.
Resolving deltas: 100% (678/678), done.


In [5]:
# Define name of project on weight and biases site
project_name = "project-3"

In [6]:
# download data
if not os.path.exists('./horse2zebra'):
    import gdown
    url = 'https://drive.google.com/uc?id=1jPelB2jzNZJq3ZU9Uk_Mkt4MJtF3DRgg'
    gdown.download(url, './horse2zebra.zip', quiet=False)
    !unzip ./horse2zebra.zip > /dev/null

# Project 3

In [7]:
# login to weight and biases
!wandb login

wandb: Currently logged in as: simonamtoft (use `wandb login --relogin` to force relogin)


In [8]:
config = {
    'batch_size': 1,
    'epochs': 10,
    'lr_d': 4*1e-4,
    'lr_g': 1*1e-4,
    'n_features': 64,
    'n_blocks': 6,
    'relu_val': 0.2,
    'img_loss': 'l2',
    'g_loss_weight': [1, 10, 5]
}

## Load Data

In [9]:
# Define data transform
transform = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor()
])

# Get images of zebras
trainset_z = ZEBRAS(dataset = "train", transform=transform)
testset_z = ZEBRAS(dataset = "test", transform=transform)

# Get images of horses
trainset_h = HORSES(dataset = "train", transform=transform)
testset_h = HORSES(dataset = "test", transform=transform)

# Convert to Dataloaders
zebra_loader = DataLoader(trainset_z, batch_size=config['batch_size'], shuffle=True)
horse_loader = DataLoader(trainset_h, batch_size=config['batch_size'], shuffle=True)

## Train Model

In [10]:
# Instantiate Cycle GAN network
d_h = Discriminator(config).to(device)
d_z = Discriminator(config).to(device)
g_h2z = Generator(config).to(device)
g_z2h = Generator(config).to(device)

# Train network
train_cycle_gan(config, g_h2z, g_z2h, d_h, d_z, zebra_loader, horse_loader, project_name)

wandb: Currently logged in as: simonamtoft (use `wandb login --relogin` to force relogin)


Epoch 1/10


KeyboardInterrupt: ignored